## Imports

In [1]:
!wget https://lodmedia.hb.bizmrg.com/case_files/993641/train_dataset_%D0%A1%D0%B8%D0%BD%D1%82%D0%B5%D0%B7.zip

--2023-08-26 07:51:57--  https://lodmedia.hb.bizmrg.com/case_files/993641/train_dataset_%D0%A1%D0%B8%D0%BD%D1%82%D0%B5%D0%B7.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 95.163.53.117
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|95.163.53.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11328990306 (11G) [application/zip]
Saving to: ‘train_dataset_Синтез.zip’

train_dataset_Синте 100%[===================>]  10.55G  9.49MB/s    in 19m 41s 

2023-08-26 08:11:39 (9.15 MB/s) - ‘train_dataset_Синтез.zip’ saved [11328990306/11328990306]



In [2]:
!unzip -q /content/train_dataset_Синтез.zip

In [3]:
!pip install mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 33.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import shutil

from tqdm.notebook import tqdm
from mediapipe_model_maker import image_classifier

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [5]:
!pip install mediapipe

In [6]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

## Create pose dataset

In [39]:
class Video():
    def __init__(self, filename):
        cap = cv2.VideoCapture(filename)
        self.filename = filename
        self.readed = False
        self.length = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        self.fps = cap.get(cv2.CAP_PROP_FPS)
        self.time = self.length/self.fps
        self.width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

        if not cap.isOpened():
            print("The file doesn't exist, or it's not a video")
            raise FileExistsError
        else:
            self.cap = cap


    def read_frame(self, n=-1):
        current_frame = self.current_frame_number()
        if n != -1:
            self.set_frame(n)

        cap = self.cap
        self.readed = True

        if cap.isOpened():
            ret, frame = cap.read()
            if n != -1:
                self.set_frame(current_frame + 1)
            return frame
        return None


    def set_frame(self, n):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, n)


    def current_frame_number(self):
        return self.cap.get(cv2.CAP_PROP_POS_FRAMES)


    def return_list(self, n=-1):
        current_frame = self.current_frame_number()
        startframe = 0

        if n != -1:
            startframe = n

        self.set_frame(startframe)
        frames = []
        frame = self.read_frame()
        # print(frame)
        # while frame[0] is not None:
        while frame is not None:
            if self.current_frame_number() == self.length:
                break
            frames.append(frame)
            frame = self.read_frame()

        self.set_frame(current_frame)

        return frames

In [14]:
pose_data = pd.read_excel('/content/positions_markup_train.xlsx')
pose_data

,Название видео,P1,P2,P3,P4,P5,P7,P8,P10
0,ab714ca7-1515-41b8-af25-70af3b04a871.mp4,4,22,23,33,42,45,46,56
1,4ea3cf67-92b9-465b-a11f-1baf8005d3c4.mp4,286,391,431,504,568,600,614,691
2,fca75c7c-0845-404c-8574-839885cb5a94.mp4,290,334,341,382,411,420,432,463
3,ea7bb598-3f31-49b3-a76b-f0edf0cc6aeb.mp4,0,14,15,26,36,41,44,56
4,62789a67-124c-47e1-92b9-77829341959f.mp4,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...
81,8f3d1f47-ad3a-4354-b26d-fba679f26032.mp4,50,64,68,76,83,88,90,101
82,27b7af84-2899-400e-acee-17833c3a122f.mp4,-,-,-,-,-,-,-,-
83,53328ebb-8501-4a56-ae2e-870a2aeac387.mp4,55,65,67,75,83,89,92,107
84,d4d18945-10e8-41cd-98c1-0777f27b621e.mp4,42,162,184,227,263,302,321,403


In [4]:
%mkdir pose_data
%cd pose_data

/content/pose_data


In [5]:
poses = pose_data.columns[1:]
for pose in poses:
    os.mkdir(pose)

%cd /content

/content


In [10]:
for i in tqdm(range(2, pose_data.shape[0])):
    row = pose_data.iloc[i]
    if '-' in row.values:
        continue
    video = Video('/content/positions_of_the_golf_swing_train/' + row[0])
    # print(video.length, video.fps)
    for pose in poses:
        frame = video.read_frame(row[pose])
        cv2.imwrite(f'/content/pose_data/{pose}/{row[0][:-4]}_{row[pose]}.jpg', frame)

  0%|          | 0/84 [00:00<?, ?it/s]

In [12]:
shutil.make_archive('pose_data', 'zip', '/content/pose_data')

'/content/pose_data.zip'

In [13]:
shutil.copy('/content/pose_data.zip', '/content/drive/MyDrive/Olymps/leadersofdigital/spb')

'/content/drive/MyDrive/Olymps/leadersofdigital/spb/pose_data.zip'

## Training

Data structure:
```
flower_photos
|__ daisy
    |______ 100080576_f52e8ee070_n.jpg
    |______ 14167534527_781ceb1b7a_n.jpg
    |______ ...
|__ dandelion
    |______ 10043234166_e6dd915111_n.jpg
    |______ 1426682852_e62169221f_m.jpg
    |______ ...
```

In [ ]:
# image_path = tf.keras.utils.get_file(
#     'flower_photos.tgz',
#     'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#     extract=True)
# image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

228813984/228813984 [==============================] - 2s 0us/step


In [14]:
%cd /content/drive/MyDrive/Olymps/leadersofdigital/spb

/content/drive/MyDrive/Olymps/leadersofdigital/spb


In [19]:
%mkdir pose_data
%cd pose_data
!unzip -q ../pose_data.zip
%cd ..

mkdir: cannot create directory ‘pose_data’: File exists
/content/drive/MyDrive/Olymps/leadersofdigital/spb/pose_data
/content/drive/MyDrive/Olymps/leadersofdigital/spb


In [20]:
image_path = '/content/drive/MyDrive/Olymps/leadersofdigital/spb/pose_data'

In [21]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, val_data = data.split(0.9)

In [25]:
spec = image_classifier.SupportedModels.EFFICIENTNET_LITE0
hparams = image_classifier.HParams(export_dir="exported_model")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [31]:
model = image_classifier.ImageClassifier.create(
    train_data=train_data,
    validation_data=val_data,
    options=options,
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              3413024   
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 8)                 10248     
                                                                 
Total params: 3423272 (13.06 MB)
Trainable params: 10248 (40.03 KB)
Non-trainable params: 3413024 (13.02 MB)
_________________________________________________________________
None
Resuming from exported_model/checkpoint/model-0010
Epoch 1/10
262/262 [==============================] - 53s 192ms/step - loss: 1.9079 - accuracy: 0.3073 - val_loss: 1.6862 - val_accuracy: 0.5085
Epoch 2/10
262/262 [==============================] - 46s 175ms/step - los

In [28]:
loss, acc = model.evaluate(val_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

2/2 [==============================] - 14s 1s/step - loss: 1.8772 - accuracy: 0.3898
Test loss:1.8772120475769043, Test accuracy:0.38983049988746643


In [32]:
model.export_model('golf_efflite_3_0.5424.tflite')

## Pose algorithm

Алгоритм определения позы
1. ждем пока вероятность Р1 станет больше остальных, отмечаем этот кадр как Р1, исключаем Р1 из возможных классов
2. ждем Р2, отмечаем Р2, исключ Р2 из возможных
...

In [24]:
def pred_frame_pose(classifier, numpy_image):
    # Load the input image from a numpy array.
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_image)
    # Perform image classification on the provided single image.
    classification_result = classifier.classify(mp_image)
    return classification_result.classifications[0].categories[0].category_name

In [25]:
def get_frames(video_path):
    video = Video(video_path)
    return video.return_list()

In [46]:
def pose_order_algo(model, video_path):
    frames = get_frames(video_path)
    phases = ['P1', 'P2', 'P3', 'P4', 'P5', 'P7', 'P8', 'P10']
    ids = []
    for i in range(0, len(frames), 2):
        # model returns 'P<num>'
        # print('pred:', pred_frame_pose(model, frames[i]))
        # print(phases[len(ids)])
        if pred_frame_pose(model, frames[i]) == phases[len(ids)]:
            ids.append(i)
            print(ids)
    if len(ids) > len(phases):
        print('ERROR', ids)
    if len(ids) < len(phases):
        ids += [len(frames)] * (len(phases) - len(ids))
    return ids

In [100]:
def pose_mean_algo(model, video_path):
    frames = get_frames(video_path)
    phases = ['P1', 'P2', 'P3', 'P4', 'P5', 'P7', 'P8', 'P10']
    phd = dict([(ph, []) for ph in phases])
    # ids = []
    for i in range(0, len(frames), 5):
        # model returns 'P<num>'
        pred = pred_frame_pose(model, frames[i])
        phd[pred] += [i]
    ids = []
    for i, pn in enumerate(list(phd.values())):
        if len(pn) == 0:
            if i == 0:
                ids.append(0)
            else:
                ids.append(ids[i - 1] + 2)
        else:
            if len(ids) == 0:
                ids.append(sum(pn) // len(pn) + 1)
            else:
                if ids[i - 1] < sum(pn) // len(pn) + 1:
                    ids.append(sum(pn) // len(pn) + 1)
                else:
                    ids.append(ids[i - 1] + 2)
    # if len(ids) > len(phases):
    #     print('ERROR', ids)
    # if len(ids) < len(phases):
    #     ids += [len(frames)] * (len(phases) - len(ids))
    return ids

## Count accuracy

In [97]:
pose_data = pd.read_excel('/content/positions_markup_train.xlsx')
pose_data = pose_data[pose_data['P1'] != '-'].reset_index(drop=True)
pose_data

,Название видео,P1,P2,P3,P4,P5,P7,P8,P10
0,ab714ca7-1515-41b8-af25-70af3b04a871.mp4,4,22,23,33,42,45,46,56
1,4ea3cf67-92b9-465b-a11f-1baf8005d3c4.mp4,286,391,431,504,568,600,614,691
2,fca75c7c-0845-404c-8574-839885cb5a94.mp4,290,334,341,382,411,420,432,463
3,ea7bb598-3f31-49b3-a76b-f0edf0cc6aeb.mp4,0,14,15,26,36,41,44,56
4,37356e9e-fc6b-40fa-a7f1-398018feee70.mp4,16,29,32,42,50,53,54,64
...,...,...,...,...,...,...,...,...,...
68,e12aa2a9-7d36-4ae4-9364-e6229836d5e9.mp4,10,24,27,39,49,52,54,67
69,e233254f-f54c-4103-9a9f-8b2bf7295e63.mp4,0,10,12,24,33,37,39,54
70,8f3d1f47-ad3a-4354-b26d-fba679f26032.mp4,50,64,68,76,83,88,90,101
71,53328ebb-8501-4a56-ae2e-870a2aeac387.mp4,55,65,67,75,83,89,92,107


In [98]:
def acc_pose(model, df):
    corr, total = 0, df.shape[0] * (df.shape[1] - 1)
    for i in tqdm(range(df.shape[0])):
        row = pose_data.iloc[i]
        y_true = row[1:].tolist()
        y_pred = pose_mean_algo(model, '/content/positions_of_the_golf_swing_train/' + row[0])
        corr += sum([abs(t - p) <= 60 for t, p in zip(y_true, y_pred)])
    return corr / total

In [101]:
acc_pose(pose_clf, pose_data)

  0%|          | 0/73 [00:00<?, ?it/s]

0.8082191780821918

In [102]:
sample = pd.read_csv('/content/submission.csv', sep=';')
sample

,Название видео,P1,P2,P3,P4,P5,P7,P8,P10
0,f8b9e19d-f454-40f9-948d-907731823033.mp4,1,1,1,1,1,1,1,1
1,a73afde3-ea62-40bb-b6e4-4008a4d8ce0a.mp4,1,1,1,1,1,1,1,1
2,6a9de841-481e-4f0f-9b47-8ca2be41891b.mp4,1,1,1,1,1,1,1,1
3,f8f1e5c4-068d-405d-8953-d640aaa08517.mp4,1,1,1,1,1,1,1,1
4,1366a9b8-ec83-4f9a-9593-235ee4623269.mp4,1,1,1,1,1,1,1,1
5,4bd7b045-a997-40e5-a60d-1b05d28d0992.mp4,1,1,1,1,1,1,1,1
6,67ae2f8c-9e3e-4215-b31b-c8b38e74a09a.mp4,1,1,1,1,1,1,1,1
7,00858b25-c5c3-4092-8ae7-5c61ada25097.mp4,1,1,1,1,1,1,1,1
8,949e17f3-2818-4815-bb1c-f26a0c95d179.mp4,1,1,1,1,1,1,1,1
9,5badc618-782b-43f2-86e6-365279981524.mp4,1,1,1,1,1,1,1,1


In [112]:
def make_submission(model, submit_path, video_path):
    sample = pd.read_csv(submit_path, sep=';')
    for i in sample.index.tolist():
        row = sample.iloc[i]
        pred = pose_mean_algo(model, video_path + row[0])
        sample.iloc[i, 1:] = pred
    sample.to_csv('fmprojects.csv')
    return sample

In [113]:
make_submission(pose_clf, '/content/submission.csv', '/content/positions_of_the_golf_swing_train/')

ZeroDivisionError: ignored

## Unused

In [13]:
pose_model_path = '/content/drive/MyDrive/Olymps/leadersofdigital/spb/exported_model/golf_efflite_3_0.5424.tflite'
base_options = python.BaseOptions(model_asset_path=pose_model_path)
options = vision.ImageClassifierOptions(
    base_options=base_options, max_results=4)
pose_clf = vision.ImageClassifier.create_from_options(options)

In [36]:
pose_data

,Название видео,P1,P2,P3,P4,P5,P7,P8,P10
0,ab714ca7-1515-41b8-af25-70af3b04a871.mp4,4,22,23,33,42,45,46,56
1,4ea3cf67-92b9-465b-a11f-1baf8005d3c4.mp4,286,391,431,504,568,600,614,691
2,fca75c7c-0845-404c-8574-839885cb5a94.mp4,290,334,341,382,411,420,432,463
3,ea7bb598-3f31-49b3-a76b-f0edf0cc6aeb.mp4,0,14,15,26,36,41,44,56
4,62789a67-124c-47e1-92b9-77829341959f.mp4,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...
81,8f3d1f47-ad3a-4354-b26d-fba679f26032.mp4,50,64,68,76,83,88,90,101
82,27b7af84-2899-400e-acee-17833c3a122f.mp4,-,-,-,-,-,-,-,-
83,53328ebb-8501-4a56-ae2e-870a2aeac387.mp4,55,65,67,75,83,89,92,107
84,d4d18945-10e8-41cd-98c1-0777f27b621e.mp4,42,162,184,227,263,302,321,403


In [ ]:
images = []
predictions = []
for image_name in IMAGE_FILENAMES:
  # STEP 3: Load the input image.
  image = mp.Image.create_from_file(image_name)

  # STEP 4: Classify the input image.
  classification_result = classifier.classify(image)

  # STEP 5: Process the classification result. In this case, visualize it.
  images.append(image)
  top_category = classification_result.classifications[0].categories[0]
  predictions.append(f"{top_category.category_name} ({top_category.score:.2f})")

display_batch_of_images(images, predictions)

In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/993641/train_dataset_%D0%A1%D0%B8%D0%BD%D1%82%D0%B5%D0%B7.zip

--2023-08-25 16:01:42--  https://lodmedia.hb.bizmrg.com/case_files/993641/train_dataset_%D0%A1%D0%B8%D0%BD%D1%82%D0%B5%D0%B7.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 95.163.53.117
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|95.163.53.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11328990306 (11G) [application/zip]
Saving to: ‘train_dataset_Синтез.zip’

train_dataset_Синте 100%[===================>]  10.55G  23.1MB/s    in 8m 2s   

2023-08-25 16:09:46 (22.4 MB/s) - ‘train_dataset_Синтез.zip’ saved [11328990306/11328990306]



In [ ]:
!unzip -q /content/train_dataset.zip

In [36]:
!gdown 1uBwRxFxW04EqG87VCoX3l6vXeV5T5JYJ

Downloading...
From: https://drive.google.com/uc?id=1uBwRxFxW04EqG87VCoX3l6vXeV5T5JYJ
To: /content/videos_160.zip
100% 699M/699M [00:07<00:00, 99.2MB/s]


In [37]:
%cd videos_160
!unzip -q /content/videos_160.zip

/content/videos_160


In [ ]:
!ls /content/videos_160/videos_160